In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import random
import matplotlib
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from surprise import Reader, Dataset
from surprise import BaselineOnly
from surprise import KNNBaseline
from surprise import SVD
from surprise import SVDpp

In [4]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 4.5 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633724 sha256=7d6b71eda32339e0b5aee65e30245736259a761755531b974725a3e6cb3d2e8c
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [35]:
#Load datafiniti hotel reviews dataset
dataset = pd.read_csv('/content/drive/MyDrive/Hotel_Recomendation_Model_Traveloka/Eksplorasi Data/Datafiniti_Hotel_Reviews_Jun19.csv')
dataset

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,...,reviews.dateSeen,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,sourceURLs,websites
0,AWE2FvX5RxPSIh2RscTK,2018-01-18T18:43:12Z,2019-05-20T23:55:47Z,5620 Calle Real,"Hotels,Hotels and motels,Hotel and motel mgmt....",Accommodation & Food Services,Goleta,US,us/ca/goleta/5620callereal/-1127060008,34.441780,...,2018-01-03T00:00:00Z,3,https://www.tripadvisor.com/Hotel_Review-g3243...,"This hotel was nice and quiet. Did not know, t...",Best Western Plus Hotel,San Jose,UnitedStates,tatsurok2018,https://www.tripadvisor.com/Hotel_Review-g3243...,https://www.bestwestern.com/en_US/book/hotel-r...
1,AVwcj_OhkufWRAb5wi9T,2016-11-06T20:21:05Z,2019-05-20T23:31:56Z,5th And San Carlos PO Box 3574,"Hotels,Lodging,Hotel",Accommodation & Food Services,Carmel by the Sea,US,us/ca/carmelbythesea/5thandsancarlospobox3574/...,36.557220,...,2016-10-09T00:00:00Z,4,https://www.tripadvisor.com/Hotel_Review-g3217...,We stayed in the king suite with the separatio...,Clean rooms at solid rates in the heart of Carmel,San Francisco,CA,STEPHEN N,http://www.tripadvisor.com/Hotel_Review-g32172...,http://www.bestwestern.com
2,AVwcj_OhkufWRAb5wi9T,2016-11-06T20:21:05Z,2019-05-20T23:31:56Z,5th And San Carlos PO Box 3574,"Hotels,Lodging,Hotel",Accommodation & Food Services,Carmel by the Sea,US,us/ca/carmelbythesea/5thandsancarlospobox3574/...,36.557220,...,2016-10-09T00:00:00Z,3,https://www.tripadvisor.com/Hotel_Review-g3217...,"Parking was horrible, somebody ran into my ren...",Business,Prescott Valley,AZ,15Deborah,http://www.tripadvisor.com/Hotel_Review-g32172...,http://www.bestwestern.com
3,AVwcj_OhkufWRAb5wi9T,2016-11-06T20:21:05Z,2019-05-20T23:31:56Z,5th And San Carlos PO Box 3574,"Hotels,Lodging,Hotel",Accommodation & Food Services,Carmel by the Sea,US,us/ca/carmelbythesea/5thandsancarlospobox3574/...,36.557220,...,2016-10-31T00:00:00Z,5,https://www.tripadvisor.com/Hotel_Review-g3217...,Not cheap but excellent location. Price is som...,Very good,Guaynabo,PR,Wilfredo M,http://www.tripadvisor.com/Hotel_Review-g32172...,http://www.bestwestern.com
4,AVwcj_OhkufWRAb5wi9T,2016-11-06T20:21:05Z,2019-05-20T23:31:56Z,5th And San Carlos PO Box 3574,"Hotels,Lodging,Hotel",Accommodation & Food Services,Carmel by the Sea,US,us/ca/carmelbythesea/5thandsancarlospobox3574/...,36.557220,...,"2016-10-09T00:00:00Z,2016-03-27T00:00:00Z",2,https://www.tripadvisor.com/Hotel_Review-g3217...,If you get the room that they advertised on th...,Low chance to come back here,Reno,NV,Luc D,http://www.tripadvisor.com/Hotel_Review-g32172...,http://www.bestwestern.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,AVwdatg0ByjofQCxo5S5,2017-04-14T05:04:33Z,2018-12-02T13:36:35Z,3101 Coliseum Dr,"Hotels,Hotels and motels,Corporate Lodging,New...",Accommodation & Food Services,Hampton,US,us/va/hampton/3101coliseumdr/-1319580369,37.057650,...,"2016-03-16T00:00:00Z,2016-04-03T00:00:00Z",4,http://www.tripadvisor.com/Hotel_Review-g57804...,My friends and I took a trip to Hampton for th...,Very accommodating and friendly staff!,Wallingford,CT,Tiffany1017,https://www.tripadvisor.com/Hotel_Review-g5780...,https://ad.doubleclick.net/ddm/clk/317271033;1...
9996,AVwdatg0ByjofQCxo5S5,2017-04-14T05:04:33Z,2018-12-02T13:36:35Z,3101 Coliseum Dr,"Hotels,Hotels and motels,Corporate Lodging,New...",Accommodation & Food Services,Hampton,US,us/va/hampton/3101coliseumdr/-1319580369,37.057650,...,2016-04-03T00:00:00Z,5,http://www.tripadvisor.com/Hotel_Review-g57804...,"from check in to departure, staff is friendly,...","comfortable, friendly, clean, professional",Homer,AK,bobg187,https://www.tripadvisor.com/Hotel_Review-g5780...,https://ad.doubleclick.net/ddm/clk/317271033;1...
9997,AVwdatg0ByjofQCxo5S5,2017-04-14T05:04:33Z,2018-12-02T13:36:35Z,3101 Coliseum Dr,"Hotels,Hotels and motels,Corporate Lodging,New...",A

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [104]:
# Take data columns hotel_id, review_username, and reviews_rating and sorting it by username
user_rating = pd.DataFrame(dataset, columns=['id', 'reviews.username', 'reviews.rating', 'reviews.text'])
user_rating = user_rating.sort_values(['reviews.username'])
user_rating

,id,reviews.username,reviews.rating,reviews.text
4954,AVwd5NMOkufWRAb5-Ldv,#1Cubsfan,4,Staff were helpful and friendly. Wireless inte...
6033,AV1Ts63z3D1zeR_xFH6n,007traveler64,5,I enjoyed a 2 night stay here recently and wil...
3211,AVwduimh_7pvs4fz8tSx,0413beanie,2,I stayed here for 2 nights. I read reviews fro...
5360,AVwdV3-D_7pvs4fz43p7,059DebDeb056,5,After a very long day of doing a college tour ...
7894,AVwcgshl_7pvs4fzv3ge,09LCM,4,Located right across the street from the Space...
...,...,...,...,...
9870,AV1Ts5IFLD2H7whiaPiu,zlbenites,5,I was here for four nights with my husband. It...
317,AWB2X-9e3-Khe5l_cVCf,zonie0531,3,The Comfort Suites Mission San Diego was clean...
5767,AVwcps_sByjofQCxgriK,zztravelerzz,4,Looks like a garden with villas but really is ...
3196,AVwc9sJZByjofQCxkFzD,zzzbill,4,This is an old Residence Inn that has been upd...


In [105]:
# Drop duplicate data id_hotel based on user_rating data
hotel_unique = user_rating.drop_duplicates(subset=['id'], keep='first')
hotel_unique

,id,reviews.username,reviews.rating,reviews.text
4954,AVwd5NMOkufWRAb5-Ldv,#1Cubsfan,4,Staff were helpful and friendly. Wireless inte...
6033,AV1Ts63z3D1zeR_xFH6n,007traveler64,5,I enjoyed a 2 night stay here recently and wil...
3211,AVwduimh_7pvs4fz8tSx,0413beanie,2,I stayed here for 2 nights. I read reviews fro...
5360,AVwdV3-D_7pvs4fz43p7,059DebDeb056,5,After a very long day of doing a college tour ...
7894,AVwcgshl_7pvs4fzv3ge,09LCM,4,Located right across the street from the Space...
...,...,...,...,...
1115,AVwclNcUIN2L1WUfmk6O,viclmzn,1,"Yesterday Sunday March 27, I reserved a one be..."
8415,AVwdKpKWkufWRAb52-u5,vinchregression,5,"I stayed during the weekend , room and bathroo..."
7678,AVwc6Hq2ByjofQCxjf93,waxapple,4,We stayed at the Best Western for a baseball t...
3635,AVzoMUvgLD2H7whiXmq_,wjbruckner,5,"The Husker Inn is a great old-style motel, wit..."


In [106]:
# Take only id_hotel
hotel_unique_id = hotel_unique['id']
hotel_unique_id

4954    AVwd5NMOkufWRAb5-Ldv
6033    AV1Ts63z3D1zeR_xFH6n
3211    AVwduimh_7pvs4fz8tSx
5360    AVwdV3-D_7pvs4fz43p7
7894    AVwcgshl_7pvs4fzv3ge
                ...         
1115    AVwclNcUIN2L1WUfmk6O
8415    AVwdKpKWkufWRAb52-u5
7678    AVwc6Hq2ByjofQCxjf93
3635    AVzoMUvgLD2H7whiXmq_
5430    AV4_H_UZ3-Khe5l_TDFN
Name: id, Length: 1433, dtype: object

In [107]:
# Drop duplicate reviews_username based on user_rating data
user_unique = user_rating.drop_duplicates(subset=['reviews.username'], keep='first')
user_unique

,id,reviews.username,reviews.rating,reviews.text
4954,AVwd5NMOkufWRAb5-Ldv,#1Cubsfan,4,Staff were helpful and friendly. Wireless inte...
6033,AV1Ts63z3D1zeR_xFH6n,007traveler64,5,I enjoyed a 2 night stay here recently and wil...
3211,AVwduimh_7pvs4fz8tSx,0413beanie,2,I stayed here for 2 nights. I read reviews fro...
5360,AVwdV3-D_7pvs4fz43p7,059DebDeb056,5,After a very long day of doing a college tour ...
7894,AVwcgshl_7pvs4fzv3ge,09LCM,4,Located right across the street from the Space...
...,...,...,...,...
9870,AV1Ts5IFLD2H7whiaPiu,zlbenites,5,I was here for four nights with my husband. It...
317,AWB2X-9e3-Khe5l_cVCf,zonie0531,3,The Comfort Suites Mission San Diego was clean...
5767,AVwcps_sByjofQCxgriK,zztravelerzz,4,Looks like a garden with villas but really is ...
3196,AVwc9sJZByjofQCxkFzD,zzzbill,4,This is an old Residence Inn that has been upd...


In [109]:
# Take only review_user_unique_data
user_unique_data = pd.DataFrame(user_unique, columns=['reviews.username', 'reviews.rating', 'reviews.text'])
user_unique_data

,reviews.username,reviews.rating,reviews.text
4954,#1Cubsfan,4,Staff were helpful and friendly. Wireless inte...
6033,007traveler64,5,I enjoyed a 2 night stay here recently and wil...
3211,0413beanie,2,I stayed here for 2 nights. I read reviews fro...
5360,059DebDeb056,5,After a very long day of doing a college tour ...
7894,09LCM,4,Located right across the street from the Space...
...,...,...,...
9870,zlbenites,5,I was here for four nights with my husband. It...
317,zonie0531,3,The Comfort Suites Mission San Diego was clean...
5767,zztravelerzz,4,Looks like a garden with villas but really is ...
3196,zzzbill,4,This is an old Residence Inn that has been upd...


In [110]:
# Step - 2

In [111]:
# Calculate count of review for each hotel
hotel_unique_total_reviews = pd.DataFrame(user_rating.groupby(['id'])['reviews.rating'].count())
hotel_unique_total_reviews

,reviews.rating
id,
AV--lQdFRxPSIh2RmtbT,69
AV--lmxKIxWefVJwsReI,1
AV--luVA3-Khe5l_ZMEJ,1
AV-AN728a4HuVbedGQnJ,90
AV-Cilg2a4HuVbedGQqK,6
...,...
AWdgUTtXa4HuVbedaODu,1
AWdlE0rK3-Khe5l_tDlq,36
AWfPeemaa4HuVbeddb2K,66


In [112]:
# Calculate Sum of Rating for each Hotel
sum_reviews_each_hotel = pd.DataFrame(user_rating.groupby(['id'])['reviews.rating'].sum())
sum_reviews_each_hotel

,reviews.rating
id,
AV--lQdFRxPSIh2RmtbT,261
AV--lmxKIxWefVJwsReI,5
AV--luVA3-Khe5l_ZMEJ,5
AV-AN728a4HuVbedGQnJ,370
AV-Cilg2a4HuVbedGQqK,26
...,...
AWdgUTtXa4HuVbedaODu,4
AWdlE0rK3-Khe5l_tDlq,139
AWfPeemaa4HuVbeddb2K,257


In [113]:
# Sort user_rating data based on hotel_id
sort_hotel_unique = hotel_unique.sort_values('id')
sort_hotel_unique

,id,reviews.username,reviews.rating,reviews.text
6123,AV--lQdFRxPSIh2RmtbT,37abhijitg,3,This is very good value for the location with ...
1552,AV--lmxKIxWefVJwsReI,foodies311,5,This was our second visit to this beautiful re...
3220,AV--luVA3-Khe5l_ZMEJ,Bill B,5,"Unfortunately, the hurricane cancelled our tri..."
5637,AV-AN728a4HuVbedGQnJ,26jrr,4,My company has a meeting here every year. Grea...
8337,AV-Cilg2a4HuVbedGQqK,Charles S,4,We enjoyed our stay at the Hampton Inn in Brem...
...,...,...,...,...
1657,AWdgUTtXa4HuVbedaODu,jackehill,4,We stayed for almost two weeks but at a condo ...
1752,AWdlE0rK3-Khe5l_tDlq,267alenkar,4,We were very satisfied with the position of th...
6652,AWfPeemaa4HuVbeddb2K,160edwardr,2,Here's the short version of my 5 nights at thi...
1919,AWfaJjQba4HuVbeddiSW,94tammyv,5,"Excellent property, great breakfast Buffett - ..."


In [114]:
# Assign new column for each hotel
id_hotel = sort_hotel_unique['id'].values
count_review_hotel = hotel_unique_total_reviews['reviews.rating'].values
sum_review_hotel = sum_reviews_each_hotel['reviews.rating'].values

average_review_hotel = sum_review_hotel / count_review_hotel

hotel = {
    'hotel_id':id_hotel,
    'count_reviews':count_review_hotel,
    'sum_reviews':sum_review_hotel,
    'average_reviews':average_review_hotel
}

new_data_hotel = pd.DataFrame(hotel)
new_data_hotel

,hotel_id,count_reviews,sum_reviews,average_reviews
0,AV--lQdFRxPSIh2RmtbT,69,261,3.782609
1,AV--lmxKIxWefVJwsReI,1,5,5.000000
2,AV--luVA3-Khe5l_ZMEJ,1,5,5.000000
3,AV-AN728a4HuVbedGQnJ,90,370,4.111111
4,AV-Cilg2a4HuVbedGQqK,6,26,4.333333
...,...,...,...,...
1428,AWdgUTtXa4HuVbedaODu,1,4,4.000000
1429,AWdlE0rK3-Khe5l_tDlq,36,139,3.861111
1430,AWfPeemaa4HuVbeddb2K,66,257,3.893939
1431,AWfaJjQba4HuVbeddiSW,2,10,5.000000


In [115]:
# Step - 3

In [116]:
# Calculate count of reviews for each user
user_unique_total_reviews = pd.DataFrame(user_rating.groupby(['reviews.username'])['reviews.rating'].count())
user_unique_total_reviews

,reviews.rating
reviews.username,
#1Cubsfan,1
007traveler64,1
0413beanie,1
059DebDeb056,1
09LCM,2
...,...
zlbenites,1
zonie0531,1
zztravelerzz,1


In [117]:
# Calculate sum of reviews for each user
sum_reviews_each_user = pd.DataFrame(user_rating.groupby(['reviews.username'])['reviews.rating'].sum())
sum_reviews_each_user

,reviews.rating
reviews.username,
#1Cubsfan,4
007traveler64,5
0413beanie,2
059DebDeb056,5
09LCM,8
...,...
zlbenites,5
zonie0531,3
zztravelerzz,4


In [118]:
# Sort user_unique_data data based on reviews.username
sort_user_unique_data = user_unique_data.sort_values('reviews.username')
sort_user_unique_data

,reviews.username,reviews.rating,reviews.text
4954,#1Cubsfan,4,Staff were helpful and friendly. Wireless inte...
6033,007traveler64,5,I enjoyed a 2 night stay here recently and wil...
3211,0413beanie,2,I stayed here for 2 nights. I read reviews fro...
5360,059DebDeb056,5,After a very long day of doing a college tour ...
7894,09LCM,4,Located right across the street from the Space...
...,...,...,...
9870,zlbenites,5,I was here for four nights with my husband. It...
317,zonie0531,3,The Comfort Suites Mission San Diego was clean...
5767,zztravelerzz,4,Looks like a garden with villas but really is ...
3196,zzzbill,4,This is an old Residence Inn that has been upd...


In [119]:
# Assign new column for each user
username = sort_user_unique_data['reviews.username'].values
count_review_user = user_unique_total_reviews['reviews.rating'].values
sum_review_user = sum_reviews_each_user['reviews.rating'].values

average_review_user = sum_review_user / count_review_user

user_reviews = {
    'username':username,
    'count_reviews':count_review_user,
    'sum_reviews':sum_review_user,
    'average_reviews':average_review_user
}

new_data_user = pd.DataFrame(user_reviews)
new_data_user

,username,count_reviews,sum_reviews,average_reviews
0,#1Cubsfan,1,4,4.0
1,007traveler64,1,5,5.0
2,0413beanie,1,2,2.0
3,059DebDeb056,1,5,5.0
4,09LCM,2,8,4.0
...,...,...,...,...
9217,zlbenites,1,5,5.0
9218,zonie0531,1,3,3.0
9219,zztravelerzz,1,4,4.0
9220,zzzbill,1,4,4.0


In [120]:
total_users = len(np.unique(user_rating["reviews.username"]))
                  
print("Total no. of Users = {}".format(total_users))

Total no. of Users = 9222


In [121]:
total_hotel = len(np.unique(user_rating["id"]))
                  
print("Total no. of Hotels = {}".format(total_hotel))

Total no. of Hotels = 1433


In [135]:
# Replace username with user_id and id with hotel_id int type
new_data_rating = user_rating.copy()
new_data_rating

,id,reviews.username,reviews.rating,reviews.text
4954,AVwd5NMOkufWRAb5-Ldv,#1Cubsfan,4,Staff were helpful and friendly. Wireless inte...
6033,AV1Ts63z3D1zeR_xFH6n,007traveler64,5,I enjoyed a 2 night stay here recently and wil...
3211,AVwduimh_7pvs4fz8tSx,0413beanie,2,I stayed here for 2 nights. I read reviews fro...
5360,AVwdV3-D_7pvs4fz43p7,059DebDeb056,5,After a very long day of doing a college tour ...
7894,AVwcgshl_7pvs4fzv3ge,09LCM,4,Located right across the street from the Space...
...,...,...,...,...
9870,AV1Ts5IFLD2H7whiaPiu,zlbenites,5,I was here for four nights with my husband. It...
317,AWB2X-9e3-Khe5l_cVCf,zonie0531,3,The Comfort Suites Mission San Diego was clean...
5767,AVwcps_sByjofQCxgriK,zztravelerzz,4,Looks like a garden with villas but really is ...
3196,AVwc9sJZByjofQCxkFzD,zzzbill,4,This is an old Residence Inn that has been upd...


In [136]:
def generateId(data, items, keyword):
  # temp = []
  # user_id = []
  # i = 0
  # for user in data.username:
  #   if user not in temp:
  #     temp.append(user)
  #     user_id.append(i+1)
  #     i += 1
  #   else:
  #     user_id.append(i)
  x = 0
  
  for item in items:
    data[keyword].replace({item: x}, inplace=True)

    x = x + 1
  
  return data
  

In [137]:
new_data_rating = generateId(new_data_rating, hotel_unique_id, 'id')
new_data_rating

,id,reviews.username,reviews.rating,reviews.text
4954,0,#1Cubsfan,4,Staff were helpful and friendly. Wireless inte...
6033,1,007traveler64,5,I enjoyed a 2 night stay here recently and wil...
3211,2,0413beanie,2,I stayed here for 2 nights. I read reviews fro...
5360,3,059DebDeb056,5,After a very long day of doing a college tour ...
7894,4,09LCM,4,Located right across the street from the Space...
...,...,...,...,...
9870,62,zlbenites,5,I was here for four nights with my husband. It...
317,143,zonie0531,3,The Comfort Suites Mission San Diego was clean...
5767,163,zztravelerzz,4,Looks like a garden with villas but really is ...
3196,509,zzzbill,4,This is an old Residence Inn that has been upd...


In [138]:
# Drop duplicate user each item review because user can't rating the same item
new_data_rating = generateId(new_data_rating, sort_user_unique_data['reviews.username'], 'reviews.username')
new_data_rating.columns = ['hotel_id', 'user_id', 'user_rating', 'user_reviews']
new_data_rating = new_data_rating.drop_duplicates(subset=['hotel_id', 'user_id'], keep='first')
new_data_rating = new_data_rating.sort_values('hotel_id')
new_data_rating.reset_index(drop=True, inplace=True)
new_data_rating

,hotel_id,user_id,user_rating,user_reviews
0,0,0,4,Staff were helpful and friendly. Wireless inte...
1,0,6853,2,Service was fine but the hotel was worn down: ...
2,0,4632,4,Hotel staff was very friendly! Room was clean ...
3,0,8325,4,The Fairfield is a value-budget hotel. Ameniti...
4,1,2065,4,Recently stayed at this property and had the p...
...,...,...,...,...
9743,1428,9093,1,"Yesterday Sunday March 27, I reserved a one be..."
9744,1429,9099,5,"I stayed during the weekend , room and bathroo..."
9745,1430,9127,4,We stayed at the Best Western for a baseball t...
9746,1431,9170,5,"The Husker Inn is a great old-style motel, wit..."


In [127]:
# Load Data Ridwan
dataset2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DataHotel_ML.csv')
dataset2

,Unnamed: 0,property_id,property_name,hotel_description,address,point_of_interest,city,state,country,latitude,...,Two Line Telephone,Utensils (on request),Video/ DVD/ CD Player Available,Voicemail.1,Wardrobe,Washing machine,Western Toilet Seat,Window Opens,Work Station,Working Desk Phone
0,0,H000001,Baragarh Regency,The standard check-in time is 12:00 PM and the...,"15th Mile, N.H.21,Manali, District Kullu,Himac...","['Hadimba Temple', 'Naggar Village', 'Himalaya...",Manali,Himachal Pradesh,India,32.139387,...,0,0,0,0,0,0,1,1,0,0
1,1,H000002,Asian Suites A- 585,The standard check-in time is 12:00 PM and the...,"A-585, Sushant Lok-1 ,Near Iffco Chowk Metro S...","['Sushant Lok', 'Sahara Mall', 'Amity Internat...",Gurgaon,Haryana,India,28.472097,...,0,1,0,0,0,0,0,0,0,0
2,2,H000003,Bevvan Resort,The standard check-in time is 12:00 PM and the...,"Cobra Vaddo,Calungate Baga Road, Bardez, Calan...","['Anjuna Beach', 'Calangute Beach', 'Titos lan...",Goa,Goa,India,15.548398,...,0,0,0,0,0,0,0,1,0,0
3,3,H000004,Apple Inn Cottage,The standard check-in time is 12:00 PM and the...,Simsa,"['The Mall', 'Tibetian Monastery', 'Vashisht H...",Manali,Himachal Pradesh,India,32.223603,...,0,0,0,0,0,0,0,0,0,0
4,4,H000005,Anmol Hotel Pvt.Ltd,The standard check-in time is 12:00 PM and the...,"8180 Street No.-6,Arakashan Road,Paharganj","['Gaffar Market', 'YMCA Institute', 'Agrasen B...",Delhi,Delhi,India,28.646777,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1948,1948,H001949,Hotel Shubh Laxmi,The standard check-in time is 12:00 PM and the...,"sarvaritu vilas , inside udaipole , opp sagasj...","['Jagdish Temple', 'City Palace', 'Udaipur Bus...",Udaipur,Rajasthan,India,24.575598,...,0,1,0,0,0,0,1,1,0,0
1949,1949,H001950,Hotel Signature,The standard check-in time is 10:00 AM and the...,"Near Yasholaxmi Nagar, Kolhapur Rd,",['Yasholaxmi Nagar'],Ichalkaranji,Maharashtra,India,16.690042,...,0,0,0,0,0,0,0,0,0,0
1950,1950,H001951,Hotel Shree Anand Restaurant &amp; Bar,The standard check-in time is 12:00 PM and the...,"Near Railway Station , Kota Jn","['Kota Airport', 'Station Road Area']",Kota,Rajasthan,India,25.222694,...,0,0,0,0,1,0,0,1,0,0
1951,1951,H001952,Hotel Silver Inn,The standard check-in time is 12:00 PM and the...,"Marol Maroshi Road, Near To Marol naka FireBri...","['Mount Mary Church', 'Juhu Beach', 'Domestic ...",Mumbai,Maharashtra,India,19.110926,...,0,0,0,0,0,0,0,0,0,0


In [128]:
# Del first column
columns = dataset2.columns.values

In [129]:
columns = np.delete(columns, 0)

In [169]:
goiboi = pd.DataFrame(dataset2, columns=columns)
goiboi = goiboi.drop(goiboi.index[1433:])
goiboi

,property_id,property_name,hotel_description,address,point_of_interest,city,state,country,latitude,longitude,...,Two Line Telephone,Utensils (on request),Video/ DVD/ CD Player Available,Voicemail.1,Wardrobe,Washing machine,Western Toilet Seat,Window Opens,Work Station,Working Desk Phone
0,H000001,Baragarh Regency,The standard check-in time is 12:00 PM and the...,"15th Mile, N.H.21,Manali, District Kullu,Himac...","['Hadimba Temple', 'Naggar Village', 'Himalaya...",Manali,Himachal Pradesh,India,32.139387,77.154660,...,0,0,0,0,0,0,1,1,0,0
1,H000002,Asian Suites A- 585,The standard check-in time is 12:00 PM and the...,"A-585, Sushant Lok-1 ,Near Iffco Chowk Metro S...","['Sushant Lok', 'Sahara Mall', 'Amity Internat...",Gurgaon,Haryana,India,28.472097,77.072546,...,0,1,0,0,0,0,0,0,0,0
2,H000003,Bevvan Resort,The standard check-in time is 12:00 PM and the...,"Cobra Vaddo,Calungate Baga Road, Bardez, Calan...","['Anjuna Beach', 'Calangute Beach', 'Titos lan...",Goa,Goa,India,15.548398,73.757634,...,0,0,0,0,0,0,0,1,0,0
3,H000004,Apple Inn Cottage,The standard check-in time is 12:00 PM and the...,Simsa,"['The Mall', 'Tibetian Monastery', 'Vashisht H...",Manali,Himachal Pradesh,India,32.223603,77.185900,...,0,0,0,0,0,0,0,0,0,0
4,H000005,Anmol Hotel Pvt.Ltd,The standard check-in time is 12:00 PM and the...,"8180 Street No.-6,Arakashan Road,Paharganj","['Gaffar Market', 'YMCA Institute', 'Agrasen B...",Delhi,Delhi,India,28.646777,77.212735,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,H001429,Tripo Sohna Road,The standard check-in time is 02:00 PM and the...,"H.No. 854,sector 38","['Emaar MGF Palm Square', 'DLF Star Mall', 'Gu...",Gurgaon,Haryana,India,28.435761,77.040615,...,0,0,0,0,0,0,0,0,0,0
1429,H001430,Treebo Singh Sons,The standard check-in time is 12:00 PM and the...,"7A / 10-11, W.E.A Channa Market, Karol Bagh","['Gaffar Market', 'Gurudwara Rakabganj Sahib',...",Delhi,Delhi,India,28.646567,77.186319,...,0,0,0,0,0,0,0,0,0,0
1430,H001431,Trident Gurgaon,The standard check-in time is 02:00 PM and the...,"443 Udyog Vihar, Phase V","['Udyog Vihar', 'Sahara Mall', 'Guru Dronachar...",Gurgaon,Haryana,India,28.501069,77.088863,...,0,0,1,1,1,0,0,0,1,0
1431,H001432,Unique Residency,The standard check-in time is 12:00 PM and the...,DLF Phase 3 S -18/1 Near Gurudhona Achariya,['Dlf Phase I I I'],Gurgaon,Haryana,India,28.613939,77.209021,...,0,0,1,0,0,0,0,0,0,0


In [170]:
# Take goiboi_id
goiboi_id = goiboi['property_id']
goiboi_id

0       H000001
1       H000002
2       H000003
3       H000004
4       H000005
         ...   
1428    H001429
1429    H001430
1430    H001431
1431    H001432
1432    H001433
Name: property_id, Length: 1433, dtype: object

In [171]:
# Take datafiniti id
datafiniti_id = new_data_rating.drop_duplicates(subset=['hotel_id'], keep='first')
datafiniti_id.reset_index(drop=True, inplace=True)
datafiniti_id = datafiniti_id['hotel_id']
datafiniti_id

0          0
1          1
2          2
3          3
4          4
        ... 
1428    1428
1429    1429
1430    1430
1431    1431
1432    1432
Name: hotel_id, Length: 1433, dtype: int64

In [172]:
# Generate Id for data hotel datafiniti
new_data_hotel = generateId(new_data_hotel, hotel_unique_id, 'hotel_id')
new_data_hotel = new_data_hotel.sort_values('hotel_id')
new_data_hotel.reset_index(drop=True, inplace=True)
new_data_hotel

,hotel_id,count_reviews,sum_reviews,average_reviews
0,0,4,14,3.500000
1,1,111,460,4.144144
2,2,4,16,4.000000
3,3,6,27,4.500000
4,4,209,899,4.301435
...,...,...,...,...
1428,1428,1,1,1.000000
1429,1429,1,5,5.000000
1430,1430,1,4,4.000000
1431,1431,1,5,5.000000


In [173]:
# Assign goiboi_id
id1 = datafiniti_id.values
id2 = goiboi_id.values
count_reviews = new_data_hotel['count_reviews'].values
sum_reviews = new_data_hotel['sum_reviews'].values
average_reviews = new_data_hotel['average_reviews'].values

merge_data = {
    'datafiniti_id':id1,
    'property_id':id2,
    'count_reviews':count_reviews,
    'sum_reviews':sum_reviews,
    'average_reviews':average_reviews
}

new_merge_data = pd.DataFrame(merge_data)
new_merge_data

,datafiniti_id,property_id,count_reviews,sum_reviews,average_reviews
0,0,H000001,4,14,3.500000
1,1,H000002,111,460,4.144144
2,2,H000003,4,16,4.000000
3,3,H000004,6,27,4.500000
4,4,H000005,209,899,4.301435
...,...,...,...,...,...
1428,1428,H001429,1,1,1.000000
1429,1429,H001430,1,5,5.000000
1430,1430,H001431,1,4,4.000000
1431,1431,H001432,1,5,5.000000


In [174]:
df_all_cols = pd.merge(new_merge_data, goiboi, how='inner', on = 'property_id')
df_all_cols

,datafiniti_id,property_id,count_reviews,sum_reviews,average_reviews,property_name,hotel_description,address,point_of_interest,city,...,Two Line Telephone,Utensils (on request),Video/ DVD/ CD Player Available,Voicemail.1,Wardrobe,Washing machine,Western Toilet Seat,Window Opens,Work Station,Working Desk Phone
0,0,H000001,4,14,3.500000,Baragarh Regency,The standard check-in time is 12:00 PM and the...,"15th Mile, N.H.21,Manali, District Kullu,Himac...","['Hadimba Temple', 'Naggar Village', 'Himalaya...",Manali,...,0,0,0,0,0,0,1,1,0,0
1,1,H000002,111,460,4.144144,Asian Suites A- 585,The standard check-in time is 12:00 PM and the...,"A-585, Sushant Lok-1 ,Near Iffco Chowk Metro S...","['Sushant Lok', 'Sahara Mall', 'Amity Internat...",Gurgaon,...,0,1,0,0,0,0,0,0,0,0
2,2,H000003,4,16,4.000000,Bevvan Resort,The standard check-in time is 12:00 PM and the...,"Cobra Vaddo,Calungate Baga Road, Bardez, Calan...","['Anjuna Beach', 'Calangute Beach', 'Titos lan...",Goa,...,0,0,0,0,0,0,0,1,0,0
3,3,H000004,6,27,4.500000,Apple Inn Cottage,The standard check-in time is 12:00 PM and the...,Simsa,"['The Mall', 'Tibetian Monastery', 'Vashisht H...",Manali,...,0,0,0,0,0,0,0,0,0,0
4,4,H000005,209,899,4.301435,Anmol Hotel Pvt.Ltd,The standard check-in time is 12:00 PM and the...,"8180 Street No.-6,Arakashan Road,Paharganj","['Gaffar Market', 'YMCA Institute', 'Agrasen B...",Delhi,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,1428,H001429,1,1,1.000000,Tripo Sohna Road,The standard check-in time is 02:00 PM and the...,"H.No. 854,sector 38","['Emaar MGF Palm Square', 'DLF Star Mall', 'Gu...",Gurgaon,...,0,0,0,0,0,0,0,0,0,0
1429,1429,H001430,1,5,5.000000,Treebo Singh Sons,The standard check-in time is 12:00 PM and the...,"7A / 10-11, W.E.A Channa Market, Karol Bagh","['Gaffar Market', 'Gurudwara Rakabganj Sahib',...",Delhi,...,0,0,0,0,0,0,0,0,0,0
1430,1430,H001431,1,4,4.000000,Trident Gurgaon,The standard check-in time is 02:00 PM and the...,"443 Udyog Vihar, Phase V","['Udyog Vihar', 'Sahara Mall', 'Guru Dronachar...",Gurgaon,...,0,0,1,1,1,0,0,0,1,0
1431,1431,H001432,1,5,5.000000,Unique Residency,The standard check-in time is 12:00 PM and the...,DLF Phase 3 S -18/1 Near Gurudhona Achariya,['Dlf Phase I I I'],Gurgaon,...,0,0,1,0,0,0,0,0,0,0


In [175]:
new_drop = df_all_cols.copy()
new_drop.drop('datafiniti_id', axis=1, inplace=True)
new_drop

,property_id,count_reviews,sum_reviews,average_reviews,property_name,hotel_description,address,point_of_interest,city,state,...,Two Line Telephone,Utensils (on request),Video/ DVD/ CD Player Available,Voicemail.1,Wardrobe,Washing machine,Western Toilet Seat,Window Opens,Work Station,Working Desk Phone
0,H000001,4,14,3.500000,Baragarh Regency,The standard check-in time is 12:00 PM and the...,"15th Mile, N.H.21,Manali, District Kullu,Himac...","['Hadimba Temple', 'Naggar Village', 'Himalaya...",Manali,Himachal Pradesh,...,0,0,0,0,0,0,1,1,0,0
1,H000002,111,460,4.144144,Asian Suites A- 585,The standard check-in time is 12:00 PM and the...,"A-585, Sushant Lok-1 ,Near Iffco Chowk Metro S...","['Sushant Lok', 'Sahara Mall', 'Amity Internat...",Gurgaon,Haryana,...,0,1,0,0,0,0,0,0,0,0
2,H000003,4,16,4.000000,Bevvan Resort,The standard check-in time is 12:00 PM and the...,"Cobra Vaddo,Calungate Baga Road, Bardez, Calan...","['Anjuna Beach', 'Calangute Beach', 'Titos lan...",Goa,Goa,...,0,0,0,0,0,0,0,1,0,0
3,H000004,6,27,4.500000,Apple Inn Cottage,The standard check-in time is 12:00 PM and the...,Simsa,"['The Mall', 'Tibetian Monastery', 'Vashisht H...",Manali,Himachal Pradesh,...,0,0,0,0,0,0,0,0,0,0
4,H000005,209,899,4.301435,Anmol Hotel Pvt.Ltd,The standard check-in time is 12:00 PM and the...,"8180 Street No.-6,Arakashan Road,Paharganj","['Gaffar Market', 'YMCA Institute', 'Agrasen B...",Delhi,Delhi,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,H001429,1,1,1.000000,Tripo Sohna Road,The standard check-in time is 02:00 PM and the...,"H.No. 854,sector 38","['Emaar MGF Palm Square', 'DLF Star Mall', 'Gu...",Gurgaon,Haryana,...,0,0,0,0,0,0,0,0,0,0
1429,H001430,1,5,5.000000,Treebo Singh Sons,The standard check-in time is 12:00 PM and the...,"7A / 10-11, W.E.A Channa Market, Karol Bagh","['Gaffar Market', 'Gurudwara Rakabganj Sahib',...",Delhi,Delhi,...,0,0,0,0,0,0,0,0,0,0
1430,H001431,1,4,4.000000,Trident Gurgaon,The standard check-in time is 02:00 PM and the...,"443 Udyog Vihar, Phase V","['Udyog Vihar', 'Sahara Mall', 'Guru Dronachar...",Gurgaon,Haryana,...,0,0,1,1,1,0,0,0,1,0
1431,H001432,1,5,5.000000,Unique Residency,The standard check-in time is 12:00 PM and the...,DLF Phase 3 S -18/1 Near Gurudhona Achariya,['Dlf Phase I I I'],Gurgaon,Haryana,...,0,0,1,0,0,0,0,0,0,0


In [176]:
new_drop.to_csv(r'/content/drive/MyDrive/Colab Notebooks/goiboi+datafiniti_merge_content-based.csv', index=False)

In [177]:
# Load Data content
dataset3 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/goiboi+datafiniti_merge_content-based.csv')
dataset3

,property_id,count_reviews,sum_reviews,average_reviews,property_name,hotel_description,address,point_of_interest,city,state,...,Two Line Telephone,Utensils (on request),Video/ DVD/ CD Player Available,Voicemail.1,Wardrobe,Washing machine,Western Toilet Seat,Window Opens,Work Station,Working Desk Phone
0,H000001,4,14,3.500000,Baragarh Regency,The standard check-in time is 12:00 PM and the...,"15th Mile, N.H.21,Manali, District Kullu,Himac...","['Hadimba Temple', 'Naggar Village', 'Himalaya...",Manali,Himachal Pradesh,...,0,0,0,0,0,0,1,1,0,0
1,H000002,111,460,4.144144,Asian Suites A- 585,The standard check-in time is 12:00 PM and the...,"A-585, Sushant Lok-1 ,Near Iffco Chowk Metro S...","['Sushant Lok', 'Sahara Mall', 'Amity Internat...",Gurgaon,Haryana,...,0,1,0,0,0,0,0,0,0,0
2,H000003,4,16,4.000000,Bevvan Resort,The standard check-in time is 12:00 PM and the...,"Cobra Vaddo,Calungate Baga Road, Bardez, Calan...","['Anjuna Beach', 'Calangute Beach', 'Titos lan...",Goa,Goa,...,0,0,0,0,0,0,0,1,0,0
3,H000004,6,27,4.500000,Apple Inn Cottage,The standard check-in time is 12:00 PM and the...,Simsa,"['The Mall', 'Tibetian Monastery', 'Vashisht H...",Manali,Himachal Pradesh,...,0,0,0,0,0,0,0,0,0,0
4,H000005,209,899,4.301435,Anmol Hotel Pvt.Ltd,The standard check-in time is 12:00 PM and the...,"8180 Street No.-6,Arakashan Road,Paharganj","['Gaffar Market', 'YMCA Institute', 'Agrasen B...",Delhi,Delhi,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,H001429,1,1,1.000000,Tripo Sohna Road,The standard check-in time is 02:00 PM and the...,"H.No. 854,sector 38","['Emaar MGF Palm Square', 'DLF Star Mall', 'Gu...",Gurgaon,Haryana,...,0,0,0,0,0,0,0,0,0,0
1429,H001430,1,5,5.000000,Treebo Singh Sons,The standard check-in time is 12:00 PM and the...,"7A / 10-11, W.E.A Channa Market, Karol Bagh","['Gaffar Market', 'Gurudwara Rakabganj Sahib',...",Delhi,Delhi,...,0,0,0,0,0,0,0,0,0,0
1430,H001431,1,4,4.000000,Trident Gurgaon,The standard check-in time is 02:00 PM and the...,"443 Udyog Vihar, Phase V","['Udyog Vihar', 'Sahara Mall', 'Guru Dronachar...",Gurgaon,Haryana,...,0,0,1,1,1,0,0,0,1,0
1431,H001432,1,5,5.000000,Unique Residency,The standard check-in time is 12:00 PM and the...,DLF Phase 3 S -18/1 Near Gurudhona Achariya,['Dlf Phase I I I'],Gurgaon,Haryana,...,0,0,1,0,0,0,0,0,0,0


In [178]:
# Assign new column for each user
id1 = datafiniti_id.values
id2 = goiboi_id.values
count_reviews = new_data_hotel['count_reviews'].values
sum_reviews = new_data_hotel['sum_reviews'].values
average_reviews = new_data_hotel['average_reviews'].values

merge_data = {
    'hotel_id':id1,
    'property_id':id2
}

new_merge_data = pd.DataFrame(merge_data)
new_merge_data

,hotel_id,property_id
0,0,H000001
1,1,H000002
2,2,H000003
3,3,H000004
4,4,H000005
...,...,...
1428,1428,H001429
1429,1429,H001430
1430,1430,H001431
1431,1431,H001432


In [181]:
df_cd = pd.merge(new_merge_data, new_data_rating, how='inner', on = 'hotel_id')
df_cd

,hotel_id,property_id,user_id,user_rating,user_reviews
0,0,H000001,0,4,Staff were helpful and friendly. Wireless inte...
1,0,H000001,6853,2,Service was fine but the hotel was worn down: ...
2,0,H000001,4632,4,Hotel staff was very friendly! Room was clean ...
3,0,H000001,8325,4,The Fairfield is a value-budget hotel. Ameniti...
4,1,H000002,2065,4,Recently stayed at this property and had the p...
...,...,...,...,...,...
9743,1428,H001429,9093,1,"Yesterday Sunday March 27, I reserved a one be..."
9744,1429,H001430,9099,5,"I stayed during the weekend , room and bathroo..."
9745,1430,H001431,9127,4,We stayed at the Best Western for a baseball t...
9746,1431,H001432,9170,5,"The Husker Inn is a great old-style motel, wit..."


In [182]:
all_merge = pd.merge(df_cd, goiboi, how='inner', on = 'property_id')
all_merge

,hotel_id,property_id,user_id,user_rating,user_reviews,property_name,hotel_description,address,point_of_interest,city,...,Two Line Telephone,Utensils (on request),Video/ DVD/ CD Player Available,Voicemail.1,Wardrobe,Washing machine,Western Toilet Seat,Window Opens,Work Station,Working Desk Phone
0,0,H000001,0,4,Staff were helpful and friendly. Wireless inte...,Baragarh Regency,The standard check-in time is 12:00 PM and the...,"15th Mile, N.H.21,Manali, District Kullu,Himac...","['Hadimba Temple', 'Naggar Village', 'Himalaya...",Manali,...,0,0,0,0,0,0,1,1,0,0
1,0,H000001,6853,2,Service was fine but the hotel was worn down: ...,Baragarh Regency,The standard check-in time is 12:00 PM and the...,"15th Mile, N.H.21,Manali, District Kullu,Himac...","['Hadimba Temple', 'Naggar Village', 'Himalaya...",Manali,...,0,0,0,0,0,0,1,1,0,0
2,0,H000001,4632,4,Hotel staff was very friendly! Room was clean ...,Baragarh Regency,The standard check-in time is 12:00 PM and the...,"15th Mile, N.H.21,Manali, District Kullu,Himac...","['Hadimba Temple', 'Naggar Village', 'Himalaya...",Manali,...,0,0,0,0,0,0,1,1,0,0
3,0,H000001,8325,4,The Fairfield is a value-budget hotel. Ameniti...,Baragarh Regency,The standard check-in time is 12:00 PM and the...,"15th Mile, N.H.21,Manali, District Kullu,Himac...","['Hadimba Temple', 'Naggar Village', 'Himalaya...",Manali,...,0,0,0,0,0,0,1,1,0,0
4,1,H000002,2065,4,Recently stayed at this property and had the p...,Asian Suites A- 585,The standard check-in time is 12:00 PM and the...,"A-585, Sushant Lok-1 ,Near Iffco Chowk Metro S...","['Sushant Lok', 'Sahara Mall', 'Amity Internat...",Gurgaon,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9743,1428,H001429,9093,1,"Yesterday Sunday March 27, I reserved a one be...",Tripo Sohna Road,The standard check-in time is 02:00 PM and the...,"H.No. 854,sector 38","['Emaar MGF Palm Square', 'DLF Star Mall', 'Gu...",Gurgaon,...,0,0,0,0,0,0,0,0,0,0
9744,1429,H001430,9099,5,"I stayed during the weekend , room and bathroo...",Treebo Singh Sons,The standard check-in time is 12:00 PM and the...,"7A / 10-11, W.E.A Channa Market, Karol Bagh","['Gaffar Market', 'Gurudwara Rakabganj Sahib',...",Delhi,...,0,0,0,0,0,0,0,0,0,0
9745,1430,H001431,9127,4,We stayed at the Best Western for a baseball t...,Trident Gurgaon,The standard check-in time is 02:00 PM and the...,"443 Udyog Vihar, Phase V","['Udyog Vihar', 'Sahara Mall', 'Guru Dronachar...",Gurgaon,...,0,0,1,1,1,0,0,0,1,0
9746,1431,H001432,9170,5,"The Husker Inn is a great old-style motel, wit...",Unique Residency,The standard check-in time is 12:00 PM and the...,DLF Phase 3 S -18/1 Near Gurudhona Achariya,['Dlf Phase I I I'],Gurgaon,...,0,0,1,0,0,0,0,0,0,0


In [183]:
new_drop2 = all_merge.copy()
new_drop2.drop('hotel_id', axis=1, inplace=True)
new_drop2

,property_id,user_id,user_rating,user_reviews,property_name,hotel_description,address,point_of_interest,city,state,...,Two Line Telephone,Utensils (on request),Video/ DVD/ CD Player Available,Voicemail.1,Wardrobe,Washing machine,Western Toilet Seat,Window Opens,Work Station,Working Desk Phone
0,H000001,0,4,Staff were helpful and friendly. Wireless inte...,Baragarh Regency,The standard check-in time is 12:00 PM and the...,"15th Mile, N.H.21,Manali, District Kullu,Himac...","['Hadimba Temple', 'Naggar Village', 'Himalaya...",Manali,Himachal Pradesh,...,0,0,0,0,0,0,1,1,0,0
1,H000001,6853,2,Service was fine but the hotel was worn down: ...,Baragarh Regency,The standard check-in time is 12:00 PM and the...,"15th Mile, N.H.21,Manali, District Kullu,Himac...","['Hadimba Temple', 'Naggar Village', 'Himalaya...",Manali,Himachal Pradesh,...,0,0,0,0,0,0,1,1,0,0
2,H000001,4632,4,Hotel staff was very friendly! Room was clean ...,Baragarh Regency,The standard check-in time is 12:00 PM and the...,"15th Mile, N.H.21,Manali, District Kullu,Himac...","['Hadimba Temple', 'Naggar Village', 'Himalaya...",Manali,Himachal Pradesh,...,0,0,0,0,0,0,1,1,0,0
3,H000001,8325,4,The Fairfield is a value-budget hotel. Ameniti...,Baragarh Regency,The standard check-in time is 12:00 PM and the...,"15th Mile, N.H.21,Manali, District Kullu,Himac...","['Hadimba Temple', 'Naggar Village', 'Himalaya...",Manali,Himachal Pradesh,...,0,0,0,0,0,0,1,1,0,0
4,H000002,2065,4,Recently stayed at this property and had the p...,Asian Suites A- 585,The standard check-in time is 12:00 PM and the...,"A-585, Sushant Lok-1 ,Near Iffco Chowk Metro S...","['Sushant Lok', 'Sahara Mall', 'Amity Internat...",Gurgaon,Haryana,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9743,H001429,9093,1,"Yesterday Sunday March 27, I reserved a one be...",Tripo Sohna Road,The standard check-in time is 02:00 PM and the...,"H.No. 854,sector 38","['Emaar MGF Palm Square', 'DLF Star Mall', 'Gu...",Gurgaon,Haryana,...,0,0,0,0,0,0,0,0,0,0
9744,H001430,9099,5,"I stayed during the weekend , room and bathroo...",Treebo Singh Sons,The standard check-in time is 12:00 PM and the...,"7A / 10-11, W.E.A Channa Market, Karol Bagh","['Gaffar Market', 'Gurudwara Rakabganj Sahib',...",Delhi,Delhi,...,0,0,0,0,0,0,0,0,0,0
9745,H001431,9127,4,We stayed at the Best Western for a baseball t...,Trident Gurgaon,The standard check-in time is 02:00 PM and the...,"443 Udyog Vihar, Phase V","['Udyog Vihar', 'Sahara Mall', 'Guru Dronachar...",Gurgaon,Haryana,...,0,0,1,1,1,0,0,0,1,0
9746,H001432,9170,5,"The Husker Inn is a great old-style motel, wit...",Unique Residency,The standard check-in time is 12:00 PM and the...,DLF Phase 3 S -18/1 Near Gurudhona Achariya,['Dlf Phase I I I'],Gurgaon,Haryana,...,0,0,1,0,0,0,0,0,0,0


In [184]:
new_drop2.to_csv(r'/content/drive/MyDrive/Colab Notebooks/goiboi+datafiniti_merge_user-collaborative-based.csv', index=False)

In [185]:
# Load Data Collaborative
dataset4 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/goiboi+datafiniti_merge_user-collaborative-based.csv')
dataset4

,property_id,user_id,user_rating,user_reviews,property_name,hotel_description,address,point_of_interest,city,state,...,Two Line Telephone,Utensils (on request),Video/ DVD/ CD Player Available,Voicemail.1,Wardrobe,Washing machine,Western Toilet Seat,Window Opens,Work Station,Working Desk Phone
0,H000001,0,4,Staff were helpful and friendly. Wireless inte...,Baragarh Regency,The standard check-in time is 12:00 PM and the...,"15th Mile, N.H.21,Manali, District Kullu,Himac...","['Hadimba Temple', 'Naggar Village', 'Himalaya...",Manali,Himachal Pradesh,...,0,0,0,0,0,0,1,1,0,0
1,H000001,6853,2,Service was fine but the hotel was worn down: ...,Baragarh Regency,The standard check-in time is 12:00 PM and the...,"15th Mile, N.H.21,Manali, District Kullu,Himac...","['Hadimba Temple', 'Naggar Village', 'Himalaya...",Manali,Himachal Pradesh,...,0,0,0,0,0,0,1,1,0,0
2,H000001,4632,4,Hotel staff was very friendly! Room was clean ...,Baragarh Regency,The standard check-in time is 12:00 PM and the...,"15th Mile, N.H.21,Manali, District Kullu,Himac...","['Hadimba Temple', 'Naggar Village', 'Himalaya...",Manali,Himachal Pradesh,...,0,0,0,0,0,0,1,1,0,0
3,H000001,8325,4,The Fairfield is a value-budget hotel. Ameniti...,Baragarh Regency,The standard check-in time is 12:00 PM and the...,"15th Mile, N.H.21,Manali, District Kullu,Himac...","['Hadimba Temple', 'Naggar Village', 'Himalaya...",Manali,Himachal Pradesh,...,0,0,0,0,0,0,1,1,0,0
4,H000002,2065,4,Recently stayed at this property and had the p...,Asian Suites A- 585,The standard check-in time is 12:00 PM and the...,"A-585, Sushant Lok-1 ,Near Iffco Chowk Metro S...","['Sushant Lok', 'Sahara Mall', 'Amity Internat...",Gurgaon,Haryana,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9743,H001429,9093,1,"Yesterday Sunday March 27, I reserved a one be...",Tripo Sohna Road,The standard check-in time is 02:00 PM and the...,"H.No. 854,sector 38","['Emaar MGF Palm Square', 'DLF Star Mall', 'Gu...",Gurgaon,Haryana,...,0,0,0,0,0,0,0,0,0,0
9744,H001430,9099,5,"I stayed during the weekend , room and bathroo...",Treebo Singh Sons,The standard check-in time is 12:00 PM and the...,"7A / 10-11, W.E.A Channa Market, Karol Bagh","['Gaffar Market', 'Gurudwara Rakabganj Sahib',...",Delhi,Delhi,...,0,0,0,0,0,0,0,0,0,0
9745,H001431,9127,4,We stayed at the Best Western for a baseball t...,Trident Gurgaon,The standard check-in time is 02:00 PM and the...,"443 Udyog Vihar, Phase V","['Udyog Vihar', 'Sahara Mall', 'Guru Dronachar...",Gurgaon,Haryana,...,0,0,1,1,1,0,0,0,1,0
9746,H001432,9170,5,"The Husker Inn is a great old-style motel, wit...",Unique Residency,The standard check-in time is 12:00 PM and the...,DLF Phase 3 S -18/1 Near Gurudhona Achariya,['Dlf Phase I I I'],Gurgaon,Haryana,...,0,0,1,0,0,0,0,0,0,0
